# Setup

In [1]:
import sys
import importlib

sys.path.insert(0, "can_reproducibility/PopV")
import popv
from popv.reproducibility import _accuracy
importlib.reload(_accuracy)

2023-02-27 16:59:18.267297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 16:59:18.349889: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-27 16:59:18.707116: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-02-27 16:59:18.707159: W tensorflow/compiler/xla/stream_executor/platf

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/PopV/popv/reproducibility/_accuracy.py'>

In [2]:
import anndata
import scanpy as sc
import numpy as np
import obonet
import networkx as nx
import pandas as pd
import time

In [3]:
from popv._utils import make_ontology_dag

In [4]:
obo_fn = 'can_reproducibility/PopV/ontology/cl.obo'

G = popv._utils.make_ontology_dag(obo_fn)

# Lung


In [5]:
fn = 'can_reproducibility/lung_models/lung_annotation/annotation/query_and_reference_popv.h5ad'
adata = anndata.read(fn)

In [6]:
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [7]:
pred_keys = adata.uns['prediction_keys']

cts = np.unique(adata.obs[pred_keys])

bad_cts = []

for ct in cts:
    if ct not in G:
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        bad_cts.append(ct)
        
cts = np.unique(adata.obs['cell_ontology_class'])

for ct in cts:
    if ct not in G:
        print('2', ct)
        bad_cts.append(ct)
    elif 'cell' not in nx.shortest_path_length(G, ct):
        print('1', ct)
        bad_cts.append(ct)

In [8]:
pred_keys

array(['popv_celltypist_prediction', 'popv_knn_on_bbknn_prediction',
       'popv_knn_on_scanorama_prediction', 'popv_knn_on_scvi_prediction',
       'popv_onclass_prediction', 'popv_rf_prediction',
       'popv_scanvi_prediction', 'popv_svm_prediction'], dtype=object)

In [9]:
# should be empty

bad_cts

[]

## Fine Accuracy

### ontology_vote

In [10]:
importlib.reload(_accuracy)

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/PopV/popv/reproducibility/_accuracy.py'>

In [11]:
paths = nx.algorithms.simple_paths.shortest_simple_paths(
                    nx.Graph(G), source='pulmonary ionocyte', target='macrophage'
                )

In [12]:
_accuracy._fine_ontology_sibling_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction')

In [13]:
adata.obs

,tissue_in_publication,donor_id,free_annotation,compartment,cell_type_ontology_term_id,cell_type,assay,sex,tissue,cell_ontology_class,...,popv_svm_prediction,popv_svm_prediction_probabilities,popv_majority_vote_prediction,popv_majority_vote_score,popv_prediction,popv_prediction_score,popv_prediction_depth,popv_prediction_onclass_relative_depth,popv_parent,popv_prediction_ontology_distance
AAACCCAAGAACTCCT_TSP14_Lung_Distal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,0.871794,type II pneumocyte,5,type II pneumocyte,5,5,-1,pneumocyte,0
TCGGGCATCACGAACT_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,0.919504,type II pneumocyte,7,type II pneumocyte,7,5,0,pneumocyte,0
TCGGGCATCTCCCTAG_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,0.893309,type II pneumocyte,8,type II pneumocyte,8,5,0,pneumocyte,0
TCGGGTGAGGTTAGTA_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,0.951251,type II pneumocyte,8,type II pneumocyte,8,5,0,pneumocyte,0
TCGGGTGGTGTTTGCA_TSP14_Lung_Proximal_10X_1_1,Lung,TSP14,type ii pneumocyte,epithelial,CL:0002063,type II pneumocyte,10x 3' v3,male,lung,type II pneumocyte,...,type II pneumocyte,0.954141,type II pneumocyte,8,type II pneumocyte,8,5,0,pneumocyte,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K23_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,"CD8-positive, alpha-beta T cell",0.462591,mature NK T cell,6,mature NK T cell,6,10,0,mature NK T cell,0
K21_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,"CD8-positive, alpha-beta T cell",0.730813,"CD8-positive, alpha-beta T cell",8,"CD8-positive, alpha-beta T cell",8,10,0,"CD8-positive, alpha-beta T cell",2_sib
B4_B002460.gencode.vH29,NaN,3,NaN,epithelial,NaN,NaN,Smart-seq2,NaN,lung,type II pneumocyte,...,type II pneumocyte,0.504444,type II pneumocyte,8,type II pneumocyte,8,5,0,pneumocyte,0
K19_B003269.gencode.vH29,NaN,3,NaN,immune,NaN,NaN,Smart-seq2,NaN,lung,mature NK T cell,...,mature NK T cell,0.515512,mature NK T cell,7,mature NK T cell,7,10,0,mature NK T cell,0


### all other methods

In [14]:
pred_keys

array(['popv_celltypist_prediction', 'popv_knn_on_bbknn_prediction',
       'popv_knn_on_scanorama_prediction', 'popv_knn_on_scvi_prediction',
       'popv_onclass_prediction', 'popv_rf_prediction',
       'popv_scanvi_prediction', 'popv_svm_prediction'], dtype=object)

In [15]:
for k in pred_keys:
    print(k)
    _accuracy._fine_ontology_sibling_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k)

popv_celltypist_prediction
popv_knn_on_bbknn_prediction
popv_knn_on_scanorama_prediction
popv_knn_on_scvi_prediction
popv_onclass_prediction
popv_rf_prediction
popv_scanvi_prediction
popv_svm_prediction


In [16]:
adata.write('can_reproducibility/lung_models/lung_annotation/annotation/query_and_reference_popv_fine_ontology_accuracy.h5ad')

## Coarse Accuracy

### ontology_vote

In [61]:
adata.obs.drop(labels=[i for i in adata.obs.columns if 'ontology_distance' in i], axis=1, inplace=True)
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [62]:
importlib.reload(_accuracy)

<module 'popv.reproducibility._accuracy' from '/data/extra_files/galen_dropbox_figures/can_reproducibility/PopV/popv/reproducibility/_accuracy.py'>

In [63]:
_accuracy._ontology_accuracy(
    adata, 
    obofile=obo_fn,
    gt_key='cell_ontology_class', 
    pred_key='popv_prediction',
    save_key=f'consensus_ontology_accuracy')

In [64]:
adata.obs['ontology_accuracy'].value_counts()

exact       94956
no match     5018
sibling      3225
parent       2122
child           1
Name: ontology_accuracy, dtype: int64

In [65]:
pd.set_option('display.max_rows', 500)

In [66]:
df = adata.obs[adata.obs['ontology_accuracy']=='no match'].copy()
df['Count'] = 1
df = df.groupby(['popv_prediction', 'cell_ontology_class']).Count.count().reset_index()
df[df['Count']>10]

,popv_prediction,cell_ontology_class,Count
55,"CD4-positive, alpha-beta T cell","effector CD8-positive, alpha-beta T cell",62
95,"CD8-positive, alpha-beta T cell","effector CD4-positive, alpha-beta T cell",1009
173,alveolar capillary type 2 endothelial cell,capillary endothelial cell,779
184,alveolar capillary type 2 endothelial cell,lung microvascular endothelial cell,12
209,basal cell,alveolar capillary type 2 endothelial cell,12
214,basal cell,capillary endothelial cell,18
216,basal cell,club cell,22
239,basal cell,respiratory goblet cell,26
268,basophil,macrophage,51
307,blood vessel endothelial cell,lung microvascular endothelial cell,13


### all other methods

In [67]:
pred_keys

array(['popv_celltypist_prediction', 'popv_knn_on_bbknn_prediction',
       'popv_knn_on_scanorama_prediction', 'popv_knn_on_scvi_prediction',
       'popv_onclass_prediction', 'popv_rf_prediction',
       'popv_scanvi_prediction', 'popv_svm_prediction'], dtype=object)

In [68]:
for k in pred_keys:
    print(k)
    _accuracy._ontology_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class', 
        pred_key=k,
        save_key=f'{k}_ontology_accuracy')

popv_celltypist_prediction
popv_knn_on_bbknn_prediction
popv_knn_on_scanorama_prediction
popv_knn_on_scvi_prediction
popv_onclass_prediction
popv_rf_prediction
popv_scanvi_prediction
popv_svm_prediction


In [69]:
adata.write('can_reproducibility/lung_models/lung_annotation/annotation/query_and_reference_popv_coarse_ontology_accuracy.h5ad')

... storing 'consensus_ontology_accuracy' as categorical
... storing 'popv_celltypist_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_bbknn_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_scanorama_prediction_ontology_accuracy' as categorical
... storing 'popv_knn_on_scvi_prediction_ontology_accuracy' as categorical
... storing 'popv_onclass_prediction_ontology_accuracy' as categorical
... storing 'popv_rf_prediction_ontology_accuracy' as categorical
... storing 'popv_scanvi_prediction_ontology_accuracy' as categorical
... storing 'popv_svm_prediction_ontology_accuracy' as categorical


# Add seurat results

In [70]:
adata = anndata.read('can_reproducibility/lung_models/lung_annotation/annotation/query_and_reference_popv_coarse_ontology_accuracy.h5ad')

In [71]:
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [72]:
df = pd.read_csv('can_reproducibility/lung_models/lung_annotation/seurat_annotation/seurat_predictions.csv')

In [73]:
df


,Unnamed: 0,predicted.id,prediction.score.type.II.pneumocyte,prediction.score.neutrophil,prediction.score.effector.CD4.positive..alpha.beta.T.cell,prediction.score.effector.CD8.positive..alpha.beta.T.cell,prediction.score.mature.NK.T.cell,prediction.score.blood.vessel.endothelial.cell,prediction.score.smooth.muscle.cell,prediction.score.adventitial.cell,...,prediction.score.bronchial.smooth.muscle.cell,prediction.score.plasmacytoid.dendritic.cell,prediction.score.club.cell,prediction.score.lung.microvascular.endothelial.cell,prediction.score.vascular.associated.smooth.muscle.cell,prediction.score.pericyte,prediction.score.mesothelial.cell,prediction.score.intermediate.monocyte,prediction.score.pulmonary.ionocyte,prediction.score.max
0,P2_1_AAACCTGAGAAACCAT,capillary endothelial cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,P2_1_AAATGCCAGATGAGAG,capillary endothelial cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,P2_1_AACACGTTCGATCCCT,capillary endothelial cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,P2_1_AACACGTTCGCACTCT,capillary endothelial cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,P2_1_AACCATGCAGCTCGCA,capillary endothelial cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69645,M2_B001772.gencode.vH29,lung ciliated cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69646,M5_B001771.gencode.vH29,lung ciliated cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69647,N2_B001769.gencode.vH29,lung ciliated cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
69648,O2_B001774.gencode.vH29,lung ciliated cell,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [74]:
df['seurat_prediction'] = df["predicted.id"]
df['seurat_probability'] = df["prediction.score.max"]

In [75]:
df.index = df["Unnamed: 0"]

In [84]:
adata.obs['seurat_prediction'] = adata.obs['cell_ontology_class']
adata.obs['seurat_probability'] = 1

In [85]:
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_prediction'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_prediction']
adata.obs.loc[adata.obs['_dataset']=='query', 'seurat_probability'] = df.loc[adata[adata.obs['_dataset']=='query'].obs_names, 'seurat_probability']

In [86]:
adata.obs['seurat_prediction']

AAACCCAAGAACTCCT_TSP14_Lung_Distal_10X_1_1      type II pneumocyte
TCGGGCATCACGAACT_TSP14_Lung_Proximal_10X_1_1    type II pneumocyte
TCGGGCATCTCCCTAG_TSP14_Lung_Proximal_10X_1_1    type II pneumocyte
TCGGGTGAGGTTAGTA_TSP14_Lung_Proximal_10X_1_1    type II pneumocyte
TCGGGTGGTGTTTGCA_TSP14_Lung_Proximal_10X_1_1    type II pneumocyte
                                                       ...        
K23_B003269.gencode.vH29                          mature NK T cell
K21_B003269.gencode.vH29                          mature NK T cell
B4_B002460.gencode.vH29                         type II pneumocyte
K19_B003269.gencode.vH29                          mature NK T cell
G3_B003269.gencode.vH29                                 basal cell
Name: seurat_prediction, Length: 105322, dtype: category
Categories (41, object): ['B cell', 'CD4-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell', 'adventitial cell', ..., 'type I pneumocyte', 'type II pneumocyte', 'vascular associated smooth musc

In [87]:
_accuracy._ontology_accuracy(
        adata, 
        obofile=obo_fn,
        gt_key='cell_ontology_class',
        pred_key='seurat_prediction',
        save_key=f'seurat_prediction_ontology_accuracy')

In [88]:
adata

AnnData object with n_obs × n_vars = 105322 × 4000
    obs: 'tissue_in_publication', 'donor_id', 'free_annotation', 'compartment', 'cell_type_ontology_term_id', 'cell_type', 'assay', 'sex', 'tissue', 'cell_ontology_class', 'cell_ontology_class_tissue', 'donor_tissue', 'assay_correct', 'donor_assay', '_batch_annotation', '_labels_annotation', '_ref_subsample', 'paper_annotation', 'sample', '_dataset', 'n_counts', 'popv_celltypist_prediction', 'popv_celltypist_prediction_probabilities', 'popv_knn_on_bbknn_prediction', 'popv_knn_on_bbknn_prediction_probabilities', 'popv_knn_on_scanorama_prediction', 'popv_knn_on_scanorama_prediction_probabilities', 'subsampled_labels', '_scvi_batch', '_scvi_labels', 'popv_knn_on_scvi_prediction', 'popv_knn_on_scvi_prediction_probabilities', '_labels_annotation_cell_ontology_id', 'popv_onclass_prediction', 'popv_onclass_seen', 'popv_onclass_prediction_probabilities', 'popv_onclass_seen_probabilities', 'popv_rf_prediction', 'popv_rf_prediction_probabilities

In [89]:
adata.write('can_reproducibility/lung_models/lung_annotation/annotation/query_and_reference_final.h5ad')

... storing 'seurat_prediction_ontology_accuracy' as categorical
